In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, classification_report
from keras.callbacks import EarlyStopping
import matplotlib.pylab as plt
import os
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, ConvLSTM2D, TimeDistributed, Bidirectional, LSTM
from keras.utils import Sequence
from tensorflow.keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf


class InferenceTimeCallback(Callback):
    def on_predict_batch_begin(self, batch, logs=None):
        self.start_time = time.time()

    def on_predict_batch_end(self, batch, logs=None):
        self.end_time = time.time()
        self.inference_time = self.end_time - self.start_time
        print(f"Inference time for batch {batch}: {self.inference_time} seconds")


In [ ]:
# Mounts drive Google

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!pip install codecarbon

### **Test sui tempi di inferenza per l'ultimo split sul modello ConvLSTM**


Partenza: clusterizzazione a 16 cluster con inizializzazione lineare dei centroidi

Prove effettuate, tale prove sono state effettuate senza "ripulire" il modello da pesi ed informazioni extra derivanti dalla clusterizzazione(indici centroidi, ...):


*   **8 Cluster e centroidi a densità**: Accuratezza = 96.1%, AUC = 99.6%, tempo medio inferenza a batch = 0.42 s, peso: 28.6 MB, consumo energetico per 102 secondi di video: 0.00123 KW/h

*   **8 Cluster e centroidi linear**: Accuratezza = 95%, AUC = 99.5%, tempo medio inferenza a batch = 0.41 s, peso: 28.6 MB, consumo energetico per 102 secondi di video: 0.0016 KW/h

*   **8 Cluster e centroidi random**: Accuratezza = 93.4%, AUC = 99.3%, tempo medio inferenza a batch = 0.41 s, peso: 28.6 MB, consumo energetico per 102 secondi di video: 0.0023 KW/h

*   **8 Cluster e centroidi Kmeans++**: Accuratezza = 95.6%, AUC = 99.6%, tempo medio inferenza a batch = 0.45 s, peso: 28.6 MB, consumo energetico per 102 secondi di video: 0.00162 KW/h

*   **16 Cluster e centroide a densità**: Accuratezza = 96.1%, AUC = 99.6%, tempo medio inferenza a batch = 0.59 s, peso: 28,6 MB, consumo energetico per 102 secondi di video: 0.0019 KW/h

*   **16 Cluster e centroidi linear**: Accuratezza = 95%, AUC = 99.5%, tempo medio inferenza a batch = 0.42 s, peso: 28,6 MB, consumo energetico per 102 secondi di video: 0.00086 KW/h

*   **16 Cluster e centroidi random**: Accuratezza = 95%, AUC = 99.46%, tempo medio inferenza a batch = 0.4 s, peso: 28.6 MB, consumo energetico per 102 secondi di video: 0.0012 KW/h

*   **16 Cluster e centroidi Kmeans++**: Accuratezza = 95%, AUC = 99.5%, tempo medio inferenza a batch = 0.42 s, peso: 28.6 MB, consumo energetico per 102 secondi di video: 0.00197 KW/h

*   **32 Cluster e centroidi a densità**: Accuratezza = 95.6%, AUC = 99.6%, tempo medio inferenza a batch = 0.4 s, peso: 28.6 MB, consumo energetico per 102 secondi di video: 0.0014 KW/h

*   **32 Cluster e centroidi linear**: Accuratezza = 95.6%, AUC = 99.6%, tempo medio inferenza a batch = 0.43 s, peso: 28.6 MB, consumo energetico per 102 secondi di video: 0.0023 KW/h

*   **32 Cluster e centroidi random**: Accuratezza = 95%, AUC = 99.5%, tempo medio inferenza a batch = 0.33 s, peso: 28.6 MB, consumo energetico per 102 secondi di video: 0.00092 KW/h

*   **32 Cluster e centroidi Kmeans++**: Accuratezza = 95.6%, AUC = 99.55%, tempo medio inferenza a batch = 0.45 s, peso: 28.6 MB, consumo energetico per 102 secondi di video: 0.0012 KW/h



In [ ]:

from tensorflow.keras.models import load_model

MobileNetV3Small_ConvLSTM= load_model('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/ConvLSTM/final_model_fold_5.h5')


MobileNetV3Small_ConvLSTM.summary()


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf




for layer in MobileNetV3Small_ConvLSTM.layers:
    weights = layer.get_weights()
    if weights:
        plt.figure(figsize=(8, 6))
        for i, w in enumerate(weights):
            plt.hist(w.flatten(), bins=50, alpha=0.7, color='blue')

        if 'time_distributed' in layer.name:
            plt.xlim(-100, 100)

        plt.title(f'Disribuzione dei pesi per {layer.name}')
        plt.xlabel('Valore dei pesi')
        plt.ylabel('Frequenza')
        plt.grid(True)
        plt.tight_layout()
        plt.show()





In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np


layer_names = []
weight_counts = []
mean_weights = []


for layer in MobileNetV3Small_ConvLSTM.layers:
    weights = layer.get_weights()
    if weights:
        total_weights = sum(w.size for w in weights)
        mean_weight = sum(w.mean() for w in weights) / len(weights)
        layer_names.append(layer.name)
        weight_counts.append(total_weights)
        mean_weights.append(mean_weight)


fig, ax = plt.subplots(figsize=(10, 8))
wedges, texts, autotexts = ax.pie(weight_counts, labels=layer_names, autopct='%1.1f%%', textprops={'weight': 'bold'})


legend_labels = [f'{layer_names[i]}: {mean_weights[i]:.2f}' for i in range(len(layer_names))]
ax.legend(wedges, legend_labels, title='Layer e Media Pesi', loc='upper right', bbox_to_anchor=(0,1,0,0))

plt.title('Distribuzione dei pesi per ciascun layer')
plt.tight_layout()
plt.show()


In [ ]:


import tensorflow as tf


for layer in MobileNetV3Small_ConvLSTM.layers:
    weights = layer.get_weights()
    if weights:
        print(f"Layer: {layer.name}")
        for i, weight in enumerate(weights):
            print(f"  Weight {i}: shape={weight.shape}, dtype={weight.dtype}")


In [ ]:
!pip install tensorflow-model-optimization

In [ ]:


import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow import keras

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

NUM_CLUSTER = 32
clustering_params = {
  'number_of_clusters': NUM_CLUSTER,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}


def custom_cluster_weights(layer):
    if isinstance(layer, keras.layers.ConvLSTM2D):
        return layer
    else:
        return cluster_weights(layer, **clustering_params)

clustered_model = keras.models.clone_model(
    MobileNetV3Small_ConvLSTM,
    clone_function=custom_cluster_weights
)

opt = keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=keras.losses.BinaryCrossentropy(),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.save("/content/gdrive/My Drive/Modelli/MobileNet_V3_Small/Clusterizzazione/ConvLSTM/clustered_model_ConvLSTM_32Kmeans.h5")

clustered_model.summary()


In [ ]:


import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time


def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)


    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches


def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


def fine_tune_model_with_videos(model, train_folder_violence, train_folder_nonviolence, input_shape, batch_size, epochs, validation_split=0.1):
    violence_videos, violence_labels = load_videos_from_folder(train_folder_violence, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(train_folder_nonviolence, 0, input_shape, batch_size)


    print(f'Number of violence video batches: {len(violence_videos)}')
    print(f'Number of non-violence video batches: {len(nonviolence_videos)}')

    all_videos = np.concatenate((violence_videos, nonviolence_videos), axis=0)
    all_labels = np.concatenate((violence_labels, nonviolence_labels), axis=0)

    print(f'Number of total video batches: {len(all_videos)}')
    print(f'Number of total labels: {len(all_labels)}')

    model.fit(
        all_videos,
        all_labels,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=validation_split
    )

    return model, all_videos, all_labels

model_path = '/content/gdrive/My Drive/Modelli/MobileNet_V3_Small/Clusterizzazione/ConvLSTM/clustered_model_ConvLSTM_32Kmeans.h5'
train_folder_violence = '/content/gdrive/MyDrive/Dataset/Data_Augmented/Violence'
train_folder_nonviolence = '/content/gdrive/MyDrive/Dataset/Data_Augmented/NonViolence'
input_shape = (224, 224)
sequence_length = 16
batch_size = 16
epochs = 1


model, all_videos, all_labels = fine_tune_model_with_videos(clustered_model, train_folder_violence, train_folder_nonviolence, input_shape, batch_size, epochs)

model.save('/content/gdrive/My Drive/Modelli/MobileNet_V3_Small/Clusterizzazione/ConvLSTM/clustered_tuned_model_ConvLSTM_32Kmeans.h5')


In [ ]:

for layer in model.layers:
    weights = layer.get_weights()
    if weights:
        plt.figure(figsize=(8, 6))
        for i, w in enumerate(weights):
            plt.hist(w.flatten(), bins=50, alpha=0.7, color='blue')

        if 'time_distributed' in layer.name:
            plt.xlim(-100, 100)

        plt.title(f'Disribuzione dei pesi per {layer.name} , cluster settati: {NUM_CLUSTER}')
        plt.xlabel('Valore dei pesi')
        plt.ylabel('Frequenza')
        plt.grid(True)
        plt.tight_layout()
        plt.show()


In [ ]:



import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import time

def load_keras_model(model_path):
    model = load_model(model_path)
    return model

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32') #/ 255.0
        frames.append(frame)

    cap.release()
    frames = np.array(frames)


    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:

        batches = []

    return batches

def run_inference(model, input_data):
    output_data = model.predict(input_data)
    return output_data

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels

from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_ConvLSTM_quantizzato_Float_pruned")
def inferenceV3_ConvLSTM_Clusterizzata():
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16


    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        start_time = time.time()
        output_data = run_inference(model, video)
        end_time = time.time()
        inference_time = end_time - start_time
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0])

    y_probs = np.array(y_probs)
    y_pred = np.round(y_probs)


    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)


    average_inference_time_per_batch = total_inference_time / len(all_videos)


    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time per Batch: {average_inference_time_per_batch:.4f} seconds')


    print('Classification Report:')
    print(classification_report(all_labels, y_pred, target_names=['NonViolent', 'Violent']))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))


    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_convLStm')

if __name__ == '__main__':
    inferenceV3_ConvLSTM_Clusterizzata()


In [ ]:
import tempfile


final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
keras.models.save_model(final_model, clustered_keras_file,
                           include_optimizer=False)


clustered_tflite_file = '/content/gdrive/My Drive/Modelli/MobileNet_V3_Small/Clusterizzazione/ConvLSTM/final_clustered_model.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)

converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]


converter._experimental_lower_tensor_list_ops = False

tflite_clustered_model = converter.convert()
with open(clustered_tflite_file, 'wb') as f:
  f.write(tflite_clustered_model)
print('Saved clustered TFLite model to:', clustered_tflite_file)


In [ ]:

import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def run_inference(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)


    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    inference_time = end_time - start_time

    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data, inference_time

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels

def inferenceV3_BiLSTM():
    model_path = 'converted_model_ConvLSTM_Quantizzazione_Int.tflite'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    interpreter = load_tflite_model(model_path)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        output_data, inference_time = run_inference(interpreter, video)
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0] / 255.0)

    y_probs = np.array(y_probs)
    print(f"Probabilità {y_probs}")
    y_pred = np.round(y_probs)



    print("Unique values in all_labels:", np.unique(all_labels))
    print("Unique values in y_pred:", np.unique(y_pred))


    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)


    average_inference_time = total_inference_time / len(all_videos)


    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time: {average_inference_time:.4f} seconds')


    print('Classification Report:')
    print(classification_report(all_labels, y_pred))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))


    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_convLStm')

if __name__ == '__main__':
    inferenceV3_BiLSTM()


### **Test sui tempi di inferenza per l'ultimo split sul modello BiLSTM**


*   **8 Cluster e centroidi a densità**: Accuratezza = 80%, AUC = 81.3%, tempo medio inferenza a batch = 0.47 s, peso: 558.4 MB, consumo energetico per 102 secondi di video: 0.0045 KW/h


*   **8 Cluster e centroidi lineari**: Accuratezza = 50.8 %, AUC = 77 %, tempo medio inferenza a batch = 0.489 s, peso: 558.4 MB, consumo energetico per 102 secondi di video: 0.004114 KW/h

*   **8 Cluster e centroidi Kmeans++**: Accuratezza = 50.8 %, AUC =76.87  %, tempo medio inferenza a batch = 0.483 s, peso: 558 MB, consumo energetico per 102 secondi di video: 0.003682 KW/h

*   **8 Cluster e centroidi Randomici**: Accuratezza = 32 %, AUC = 26 %, tempo medio inferenza a batch = 0.546 s, peso: 558 MB, consumo energetico per 102 secondi di video: 0.004158  KW/h

*   **16 Cluster e centroide a densità**: Accuratezza = 53 %, AUC = 81.7 %, tempo medio inferenza a batch = 0.7550 s, peso: 558 MB, consumo energetico per 102 secondi di video: 0.005625 kWh KW/h



In [ ]:

from tensorflow.keras.models import load_model

MobileNetV3Small_BiLSTM= load_model('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/BiLSTM/final_model_fold_5.h5')


MobileNetV3Small_BiLSTM.summary()

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf




for layer in MobileNetV3Small_BiLSTM.layers:
    weights = layer.get_weights()
    if weights:
        plt.figure(figsize=(8, 6))
        for i, w in enumerate(weights):
            plt.hist(w.flatten(), bins=50, alpha=0.7, color='blue')

        if 'time_distributed' in layer.name:
            plt.xlim(-100, 100)

        plt.title(f'Disribuzione dei pesi per {layer.name}')
        plt.xlabel('Valore dei pesi')
        plt.ylabel('Frequenza')
        plt.grid(True)
        plt.tight_layout()
        plt.show()



In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np


layer_names = []
weight_counts = []
mean_weights = []


time_distributed_count = 0
bidirectional_count = 0


for layer in MobileNetV3Small_BiLSTM.layers:
    weights = layer.get_weights()
    if weights:
        total_weights = sum(w.size for w in weights)
        mean_weight = sum(w.mean() for w in weights) / len(weights)
        layer_names.append(layer.name)
        weight_counts.append(total_weights)
        mean_weights.append(mean_weight)


        if isinstance(layer, tf.keras.layers.TimeDistributed):
            time_distributed_count += total_weights
        elif isinstance(layer, tf.keras.layers.Bidirectional):
            bidirectional_count += total_weights


print(f'Totale pesi nel layer TimeDistributed: {time_distributed_count}')
print(f'Totale pesi nel layer Bidirectional: {bidirectional_count}')


fig, ax = plt.subplots(figsize=(10, 8))
wedges, texts, autotexts = ax.pie(weight_counts, labels=layer_names, autopct='%1.1f%%', textprops={'weight': 'bold'})


plt.tight_layout()


legend_labels = [f'{layer_names[i]}: {mean_weights[i]:.2f}' for i in range(len(layer_names))]
ax.legend(wedges, legend_labels, title='Layer e Media Pesi', loc='upper right', bbox_to_anchor=(1, 1))

plt.title('Distribuzione dei pesi per ciascun layer')
plt.show()


In [ ]:


import tensorflow as tf


for layer in MobileNetV3Small_BiLSTM.layers:
    weights = layer.get_weights()
    if weights:
        print(f"Layer: {layer.name}")
        for i, weight in enumerate(weights):
            print(f"  Weight {i}: shape={weight.shape}, dtype={weight.dtype}")


In [ ]:
!pip install tensorflow-model-optimization

In [ ]:


import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow import keras

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

NUM_CLUSTER = 16
clustering_params = {
  'number_of_clusters': NUM_CLUSTER,
  'cluster_centroids_init': CentroidInitialization.DENSITY_BASED
}


def custom_cluster_weights(layer):
    if isinstance(layer, keras.layers.ConvLSTM2D):
        return layer
    else:
        return cluster_weights(layer, **clustering_params)


clustered_model = keras.models.clone_model(
    MobileNetV3Small_BiLSTM,
    clone_function=custom_cluster_weights
)

opt = keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=keras.losses.BinaryCrossentropy(),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.save("/content/gdrive/My Drive/Modelli/MobileNet_V3_Small/Clusterizzazione/BiLSTM/clustered_model_BiLSTM_8Density.h5")

clustered_model.summary()


In [ ]:


import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time


def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)


    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches


def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


def fine_tune_model_with_videos(model, train_folder_violence, train_folder_nonviolence, input_shape, batch_size, epochs, validation_split=0.1):
    violence_videos, violence_labels = load_videos_from_folder(train_folder_violence, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(train_folder_nonviolence, 0, input_shape, batch_size)


    print(f'Number of violence video batches: {len(violence_videos)}')
    print(f'Number of non-violence video batches: {len(nonviolence_videos)}')

    all_videos = np.concatenate((violence_videos, nonviolence_videos), axis=0)
    all_labels = np.concatenate((violence_labels, nonviolence_labels), axis=0)

    print(f'Number of total video batches: {len(all_videos)}')
    print(f'Number of total labels: {len(all_labels)}')

    model.fit(
        all_videos,
        all_labels,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=validation_split
    )

    return model, all_videos, all_labels

model_path = '/content/gdrive/My Drive/Modelli/MobileNet_V3_Small/Clusterizzazione/BiLSTM/clustered_model_BiLSTM_8Density.h5'
train_folder_violence = '/content/gdrive/MyDrive/Dataset/Data_Augmented/Violence'
train_folder_nonviolence = '/content/gdrive/MyDrive/Dataset/Data_Augmented/NonViolence'
input_shape = (224, 224)
sequence_length = 16
batch_size = 16
epochs = 1


model, all_videos, all_labels = fine_tune_model_with_videos(clustered_model, train_folder_violence, train_folder_nonviolence, input_shape, batch_size, epochs)

model.save('/content/gdrive/My Drive/Modelli/MobileNet_V3_Small/Clusterizzazione/BiLSTM/clustered_tuned_model_BiLSTM_8Density.h5')


In [ ]:
model_quantized_size = os.path.getsize('/content/gdrive/My Drive/Modelli/MobileNet_V3_Small/Clusterizzazione/BiLSTM/clustered_tuned_model_BiLSTM_8Density.h5')
model_quantized_size_mb = model_quantized_size / (1024 * 1024)
print(f"/content/gdrive/My Drive/Modelli/MobileNet_V3_Small/Clusterizzazione/BiLSTM/clustered_tuned_model_BiLSTM_8Density.h5'  Peso modello: {model_quantized_size_mb} MB")

In [ ]:

for layer in model.layers:
    weights = layer.get_weights()
    if weights:
        plt.figure(figsize=(8, 6))
        for i, w in enumerate(weights):
            plt.hist(w.flatten(), bins=50, alpha=0.7, color='blue')

        if 'time_distributed' in layer.name:
            plt.xlim(-100, 100)

        plt.title(f'Disribuzione dei pesi per {layer.name} , cluster settati: {NUM_CLUSTER}')
        plt.xlabel('Valore dei pesi')
        plt.ylabel('Frequenza')
        plt.grid(True)
        plt.tight_layout()
        plt.show()


In [ ]:



import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import time

def load_keras_model(model_path):
    model = load_model(model_path)
    return model

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)


    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:

        batches = []

    return batches

def run_inference(model, input_data):
    output_data = model.predict(input_data)
    return output_data

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels

from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_ConvLSTM_quantizzato_Float_pruned")
def inferenceV3_BiLSTM_Clusterizzata():
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16


    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        start_time = time.time()
        output_data = run_inference(model, video)
        end_time = time.time()
        inference_time = end_time - start_time
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0])

    y_probs = np.array(y_probs)
    y_pred = np.round(y_probs)


    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)


    average_inference_time_per_batch = total_inference_time / len(all_videos)


    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time per Batch: {average_inference_time_per_batch:.4f} seconds')


    print('Classification Report:')
    print(classification_report(all_labels, y_pred, target_names=['NonViolent', 'Violent']))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))


    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_BiLStm')

if __name__ == '__main__':
    inferenceV3_BiLSTM_Clusterizzata()
